In [8]:
import json
import pandas as pd

df_vermont = pd.read_json("review-Vermont_10.json", lines=True, orient='columns')
df_del = pd.read_json('review-Delaware_10.json', lines=True, orient='columns')
df_dc = pd.read_json('review-District_of_Columbia_10.json', lines=True, orient='columns')
df_ri = pd.read_json('review-Rhode_Island_10.json', lines=True, orient='columns')

In [10]:
# Drop reviews without text
df_del = df_del[df_del['text'].astype(bool)]
df_vermont = df_vermont[df_vermont['text'].astype(bool)]
df_dc = df_dc[df_dc['text'].astype(bool)]
df_ri = df_ri[df_ri['text'].astype(bool)]

In [24]:
# Combine into one dataframe
df = pd.concat([df_del[['rating', 'text']].copy(), 
                df_vermont[['rating', 'text']].copy(), 
                df_dc[['rating', 'text']].copy(), 
                df_ri[['rating', 'text']].copy()])

In [32]:
df.groupby(by='rating').count()

,text
rating,
1,71825
2,52628
3,129414
4,310615
5,869607


In [34]:
df.to_csv('review-text.csv')

In [48]:
# Select equal number of each rating review (done randomly)
df_1s = df[df['rating'] == 1]
df_2s = df[df['rating'] == 2]
df_3s = df[df['rating'] == 3]
df_4s = df[df['rating'] == 4]
df_5s = df[df['rating'] == 5]

df_equal_counts = pd.concat([df_1s.sample(n=50000).copy(), 
                             df_2s.sample(n=50000).copy(), 
                             df_3s.sample(n=50000).copy(),
                             df_4s.sample(n=50000).copy(),
                             df_5s.sample(n=50000).copy()
                            ])

In [180]:
## TRIAL - there's better ways to do this

## Bag of words model -- remove all non text characters (not including space)
## Also forces all reviews to be lowercase text only

alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ 1234567890')

def clean_text(txt:str):
    # People who write with ellipses?
    txt.replace('(Translated by Google)', '')
    txt.replace('....', ' ')
    txt.replace('...', ' ')
    txt.replace('..', ' ')
    txt.replace('\n', ' ')
    txt.replace('  ', ' ')

    # Replace other characters
    for char in txt:
        if char not in alphabet:
            txt = txt.replace(char, '')
    return txt.lower()
    

In [182]:
df_equal_counts['cleaned'] = df_equal_counts['text'].apply(clean_text)

In [196]:
df_equal_counts.sample(n=12)

,rating,text,cleaned
85902,2,They have a wide range of items but they are v...,they have a wide range of items but they are v...
91627,3,It's mallarific!,its mallarific
167356,5,This Costco is the best because in addition to...,this costco is the best because in addition to...
386606,1,Got pulled forward and sat there for almost 30...,got pulled forward and sat there for almost 30...
271875,3,Everything here is overpriced to the point of ...,everything here is overpriced to the point of ...
182411,5,Really great staff. Straight answers. Worth ev...,really great staff straight answers worth ever...
833394,4,One of the best chains. Wait time very long,one of the best chains wait time very long
313648,1,Quality low. They put what they want on burger...,quality low they put what they want on burgers...
98051,3,Good music... Nice atmosphere...can get crowde...,good music nice atmospherecan get crowded at t...
321856,3,Food was great but service was a little slow.,food was great but service was a little slow


In [188]:
df_equal_counts.to_csv('review-text-equal-count.csv')